## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
from keras.models import Model
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense,  Input
from tensorflow.keras.optimizers import Adam

# Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')

# View the first five rows of data
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
print(attrition_df.columns)

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')


In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [5]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Age', 'BusinessTravel', 'DistanceFromHome', 'Education', 'JobRole', 
             'EnvironmentSatisfaction', 'JobSatisfaction', 'OverTime', 
             'NumCompaniesWorked', 'YearsAtCompany']


# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes


Age                         int64
BusinessTravel             object
DistanceFromHome            int64
Education                   int64
JobRole                    object
EnvironmentSatisfaction     int64
JobSatisfaction             int64
OverTime                   object
NumCompaniesWorked          int64
YearsAtCompany              int64
dtype: object

In [6]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)



In [7]:
# Verify if 'NumCompaniesWorked' is still present in the training data
print(X_train.columns)

Index(['Age', 'BusinessTravel', 'DistanceFromHome', 'Education', 'JobRole',
       'EnvironmentSatisfaction', 'JobSatisfaction', 'OverTime',
       'NumCompaniesWorked', 'YearsAtCompany'],
      dtype='object')


In [8]:
# Clean column names by stripping any leading/trailing spaces
X_train.columns = X_train.columns.str.strip()
X_test.columns = X_test.columns.str.strip()

# Recheck if 'NumCompaniesWorked' is present
print(X_train.columns)

Index(['Age', 'BusinessTravel', 'DistanceFromHome', 'Education', 'JobRole',
       'EnvironmentSatisfaction', 'JobSatisfaction', 'OverTime',
       'NumCompaniesWorked', 'YearsAtCompany'],
      dtype='object')


In [9]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
# Use ColumnTransformer to apply OneHotEncoding to categorical features and keep numerical columns as is
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', ['Age', 'DistanceFromHome', 'NumCompaniesWorked', 'YearsAtCompany']),
        ('cat', OneHotEncoder(), ['BusinessTravel', 'Education', 'JobRole', 'EnvironmentSatisfaction', 
                                  'JobSatisfaction', 'OverTime'])
    ])

# Fit and transform the training data, transform the testing data
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)


In [10]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train_transformed)

# Scale the training and testing data
X_test_scaled = scaler.transform(X_test_transformed)


In [11]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data


# Create two new variables by applying the encoder
# to the training and testing data
department_train = department_encoder.fit_transform(y_train[['Department']])
department_test = department_encoder.transform(y_test[['Department']])

# Check the shape of department_train to ensure it matches the model output
#print(department_train.shape)  # Should be (n_samples, 3)
print(f"Shape of department_train: {department_train.shape}")  # Should be (n_samples, num_classes)
print(f"Shape of department_test: {department_test.shape}")    # Should be (n_samples, num_classes)

Shape of department_train: (1176, 3)
Shape of department_test: (294, 3)


In [12]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, drop=None)


# Fit the encoder to the training data
attrition_train = attrition_encoder.fit_transform(y_train[['Attrition']])
attrition_test = attrition_encoder.transform(y_test[['Attrition']])


# Create two new variables by applying the encoder
# to the training and testing data
# attrition_train = attrition_encoder.fit_transform(y_train[['Attrition']])
# attrition_test = attrition_encoder.transform(y_test[['Attrition']])
# Check the result of the transformation
print(attrition_train[:5])  # View the first 5 rows of the transformed training data
print(attrition_test[:5])   # View the first 5 rows of the transformed testing data
print(attrition_train.shape)
print(attrition_train.shape)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]
(1176, 2)
(1176, 2)


In [13]:
print(attrition_train.shape)  # Should be (n_samples, 2) for binary classification
print(attrition_test.shape)   # Should match the training shape

(1176, 2)
(294, 2)


## Create, Compile, and Train the Model

In [14]:
# Find the number of columns in the X training data
#num_features = X_train_scaled.shape[1]

#print(f'Number of input features: {input_dim}')

# Create the input layer
#input_layer = Input(shape=(num_features,))
input_layer = Input(shape=(31,))

# Create at least two shared layers
# shared_layer_1 = Dense(64, activation='relu')(input_layer)
# shared_layer_2 = Dense(32, activation='relu')(shared_layer_1)
# shared_layer_1 = Dense(128, activation='relu')(input_layer)
# shared_layer_2 = Dense(64, activation='relu')(shared_layer_1)
dense1 = Dense(128, activation='relu')(input_layer)
dense2 = Dense(64, activation='relu')(dense1)



In [15]:
print(department_train.shape[1])

3


In [ ]:
import tensorflow as tf
print(tf.__version__)

In [16]:
# Create a branch for Department
# with a hidden layer and an output layer
#department_hidden = Dense(16, activation='relu')(shared_layer_2)
#dept_branch = Dense(32, activation='relu')(shared_layer_2)
#dept_output = Dense(department_train.shape[1], activation='softmax', name='dept_output')(dept_branch)
dept_output = Dense(3, activation='softmax', name='dept_output')(dense2)

# Create the hidden layer


# Create the output layer
#department_output = Dense(department_train.shape[1], activation='softmax', name='department_output')(department_hidden)


In [17]:
# Create a branch for Attrition
# with a hidden layer and an output layer
#attrition_hidden = Dense(16, activation='relu')(shared_layer_2)
#attr_branch = Dense(32, activation='relu')(shared_layer_2)
#attr_output = Dense(2, activation='softmax', name='attr_output')(attr_branch)
attr_output = Dense(2, activation='softmax', name='attr_output')(dense2)

# Create the hidden layer


# Create the output layer

#attrition_output = Dense(attrition_train.shape[1], activation='softmax', name='attrition_output')(attrition_hidden)


In [27]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attr_output])

# Compile the model
# If necessary, reinitialize the model
#model = ...  # Recreate your model architecture
model.compile(optimizer='adam', 
              loss={'dept_output': 'categorical_crossentropy', 
                    'attr_output': 'categorical_crossentropy'},
              metrics={'dept_output': 'accuracy', 
                       'attr_output': 'accuracy'})
# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 31)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │      4,096 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_output (Dense) │ (None, 3)         │        195 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attr_output (Dense) │ (None, 2)         │        130 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,677 (49.52 KB)

 Trainable params: 12,677 (49.52 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
print(f"department_train shape: {department_train.shape}")
print(f"attrition_train shape: {attrition_train.shape}")

department_train shape: (1176, 3)
attrition_train shape: (1176, 2)


In [20]:
print(f"department_test shape: {department_test.shape}")
print(f"attrition_test shape: {attrition_test.shape}")

department_test shape: (294, 3)
attrition_test shape: (294, 2)


In [21]:
output_shapes = model.outputs
for output in output_shapes:
    print(f"Output shape: {output.shape}")

Output shape: (None, 3)
Output shape: (None, 2)


In [28]:
# Train the model
history = model.fit(X_train_scaled, 
                    {'dept_output': department_train, 'attr_output': attrition_train},
                    epochs=50, 
                    batch_size=32, 
                    validation_data=(X_test_scaled, 
                                     {'dept_output': department_test, 'attr_output': attrition_test}),
                    verbose=1)

Epoch 1/50


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 2), output.shape=(None, 3)

In [23]:
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"department_train shape: {department_train.shape}")
print(f"attrition_train shape: {attrition_train.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")
print(f"department_test shape: {department_test.shape}")
print(f"attrition_test shape: {attrition_test.shape}")

X_train_scaled shape: (1176, 31)
department_train shape: (1176, 3)
attrition_train shape: (1176, 2)
X_test_scaled shape: (294, 31)
department_test shape: (294, 3)
attrition_test shape: (294, 2)


In [25]:
print(department_train[:5])
print(attrition_train[:5])

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [24]:
# Evaluate the model with the testing data
evaluation = model.evaluate(X_test_scaled, {'dept_output': department_test, 'attr_output': attrition_test})
print(f"Test Loss: {evaluation[0]}")
print(f"Department Accuracy: {evaluation[1]}")
print(f"Attrition Accuracy: {evaluation[2]}")

ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 2), output.shape=(None, 3)

In [ ]:
# Print the accuracy for both department and attrition


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 